#### 步骤1：导入依赖库

In [99]:
import math
import numpy as np
import pandas as pd
import os
import random
import codecs
from pathlib import Path
import evaluate

import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
from tqdm import tqdm

#### 步骤2：设置各种参数
`PromptTuningConfig`：`token`数量

训练参数：学习率，最大长度

In [100]:
device = "cuda"
model_name_or_path = "openai-community/gpt2"
tokenizer_name_or_path = "openai-community/gpt2"
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    prompt_tuning_init_text="Classify whether the review is positive or negative:",
    tokenizer_name_or_path=model_name_or_path,
)

dataset_name = "movie_review"
checkpoint_name = f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
    "/", "_"
)
text_column = "text"
label_column = "label"
max_length = 64
lr = 3e-2
num_epochs = 20
batch_size = 64

#### 步骤3：数据集处理

In [101]:
dataset = load_dataset("./data/movie_review")

classes = [k for k in dataset["train"]["label"]]
print(dataset)
print(dataset["train"][0])
print(dataset["test"][0])

Found cached dataset csv (/home/ma-user/.cache/huggingface/datasets/csv/movie_review-9e5381feaf6d4c1d/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 2/2 [00:00<00:00, 821.29it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8528
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2133
    })
})
{'text': 'both an admirable reconstruction of terrible events , and a fitting memorial to the dead of that day , and of the thousands thereafter .', 'label': 'positive'}
{'text': 'at its best ( and it does have some very funny sequences ) looking for leonard reminds you just how comically subversive silence can be .', 'label': 'positive'}


#### 步骤4：模型初始化
加载预训练模型和分词器，并获取该模型的嵌入层

In [102]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])
print(target_max_length)

1


In [103]:
print(len(dataset["train"][text_column]))

8528


In [104]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets, add_special_tokens=False)  # don't add bos token because we concatenate with inputs
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [105]:
train_dataset = dataset["train"].map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

eval_dataset = dataset["test"].map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["test"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

#### 步骤5：模型和优化器设置
加载预训练模型，将软提示词嵌入层集成到模型中，并定义损失函数和优化器。

In [106]:
# creating model
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
# model
# optimizer and lr scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

trainable params: 6144 || all params: 124445952 || trainable%: 0.00493708304790822


#### 步骤6：训练过程
定义训练过程，优化软提示以最小化训练数据上的损失。

In [107]:
# metric = evaluate.load("./metrics/mse")
# metric.features

In [108]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    # eval_labels = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"epoch: {epoch}\ntrain_ppl: {train_ppl}, train_epoch_loss: {train_epoch_loss}\neval_ppl: {eval_ppl}, eval_epoch_loss: {eval_epoch_loss}")

100%|██████████| 34/34 [00:04<00:00,  7.69it/s]


epoch: 0
train_ppl: 2.2827303409576416, train_epoch_loss: 0.8253722190856934
eval_ppl: 1.3641374111175537, eval_epoch_loss: 0.3105222284793854


100%|██████████| 34/34 [00:04<00:00,  7.61it/s]


epoch: 1
train_ppl: 1.3889670372009277, train_epoch_loss: 0.32856041193008423
eval_ppl: 1.2928080558776855, eval_epoch_loss: 0.2568165957927704


100%|██████████| 34/34 [00:04<00:00,  7.61it/s]


...

epoch: 17
train_ppl: 1.234417200088501, train_epoch_loss: 0.21059897541999817
eval_ppl: 1.2027664184570312, eval_epoch_loss: 0.18462423980236053


100%|██████████| 34/34 [00:04<00:00,  7.56it/s]


epoch: 18
train_ppl: 1.2294172048568726, train_epoch_loss: 0.20654018223285675
eval_ppl: 1.2036818265914917, eval_epoch_loss: 0.185385063290596


100%|██████████| 34/34 [00:04<00:00,  7.54it/s]

epoch: 19
train_ppl: 1.2314279079437256, train_epoch_loss: 0.20817437767982483
eval_ppl: 1.203476071357727, eval_epoch_loss: 0.1852140575647354


#### 步骤7：测试评估
评估软提示在测试数据上的表现。

In [109]:
model.eval()
i = 33
inputs = tokenizer(f'{text_column} : {dataset["test"][i]["text"]} label : ', return_tensors="pt")
print(dataset["test"][i]["text"])
print(inputs)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=10, eos_token_id=3
    )
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tells a fascinating , compelling story .
{'input_ids': tensor([[ 5239,  1058,  4952,   257, 13899,   837, 13206,  1621,   764,  6167,
          1058,   220]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[ 5239,  1058,  4952,   257, 13899,   837, 13206,  1621,   764,  6167,
          1058,   220, 24561, 50256,   464,  1708,   318,   257,  1351,   286,
           262,   749]], device='cuda:0')
['text : tells a fascinating, compelling story. label : positiveThe following is a list of the most']


#### 步骤8：保存模型

In [110]:
# saving model
peft_model_id = f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}".replace(
    "/", "_"
)
model.save_pretrained(peft_model_id)
ckpt = f"{peft_model_id}/adapter_model.bin"